In [66]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

df = pd.read_csv('data/table_matches_new.csv')
df = df.drop(columns=['Unnamed: 0'])
count = 1736219

while count != 1737122 :
    try:
        count += 1
        if (count % 2) == 0:
            sleep(1)
        sleep(1)
        url = f"https://soccer365.ru/games/{count}/"

        response = requests.get(url)

        soup = BeautifulSoup(response.text,'lxml')

        title = ['id',"общая информация","Дата",'Команда1',"счет1",'Команда2',"счет2","xG1","xG2","Удары1","Удары2",'Удары в створ1','Удары в створ2','Сейвы1','Сейвы2','Владение %1','Владение %2','Угловые1','Угловые2','Нарушения1','Нарушения2','Офсайды1','Офсайды2','Желтые карточки1','Желтые карточки2','Красные карточки1','Красные карточки2','Атаки1','Атаки2','Опасные атаки1','Опасные атаки2','Передачи1','Передачи2','Точность передач %1','Точность передач %2','Штрафные удары1','Штрафные удары2','Вбрасывания1','Вбрасывания2','Навесы1','Навесы2','Температура',"Признак погоды",'дата ПМ1','Пголы1',"Пза матч1","Ппобед1","Пничьи1","Ппорож1",'дата ПМ2','Пголы2',"Пза матч2","Ппобед2","Пничьи2","Ппорож2","поб1","X","поб2","ТМ 2.5","ТБ 2.5"]
        lister = list()
        lister.append(count)
        #Общая инфа и время матча
        data = soup.find("div",class_="block_header bkcenter").text.replace("\n","")
        date_list = data.split()
        date_time = date_list[-2] + ' ' + date_list[-1]
        lister.append(data)
        lister.append(date_time)

        #Название команд и счет

        result_match_1= soup.find("div",class_="live_game left")
        result_match_2= soup.find("div",class_="live_game right")

        name_1_com = result_match_1.find("div",class_="live_game_ht").text.replace("\n","")
        goal_1_com = result_match_1.find("div",class_="live_game_goal").text.replace("\n","")

        name_2_com = result_match_2.find("div",class_="live_game_at").text.replace("\n","")
        goal_2_com = result_match_2.find("div",class_="live_game_goal").text.replace("\n","")
        lister.append(name_1_com)
        lister.append(goal_1_com)
        lister.append(name_2_com)
        lister.append(goal_2_com)

        #основные  данные о матче
        gm_block= soup.find("div",class_="block_body")

        result_matches_1 = gm_block.find_all("div",class_="stats_items")[0]
        result_table_matches_1 = result_matches_1.find_all("div", class_="stats_inf")
        for i in (result_table_matches_1):
            lister.append(float(i.text))


        result_matches_2 = gm_block.find_all("div",class_="stats_items")[1]
        result_table_matches_2 = result_matches_2.find_all("div", class_="stats_inf")
        if len(result_table_matches_2) == 16:
            for j in (result_table_matches_2):
                lister.append(float(j.text))
        else :
            result_table_matches_2 = result_table_matches_2[0:16]
            for j in (result_table_matches_2):
                lister.append(float(j.text))
            
        #Погода
        preview_st = soup.find("div",class_="preview_item st")
        temper = preview_st.find("span",class_="red").text

        priznac_pog = preview_st.find_all("span",class_="min_gray")[1].text
        lister.append(temper)
        lister.append(priznac_pog)

        #инфа за прошлые 5 матчей
        preview_st = soup.find("div",class_="gm_block inline-block")
        home_prewie = preview_st.find('div',class_='live_block_hf')
        home_table = home_prewie.find('table',class_="game_form_team_stats")

        home_old_match = home_prewie.find('span', class_="size10")
        lister.append(home_old_match.text)

        home_table_text = home_table.find_all('td')[1:6]
        for i in (home_table_text):
            lister.append(float(i.text))



        atac_prewie = preview_st.find('div',class_='live_block_hf right')
        atac_table = atac_prewie.find('table',class_="game_form_team_stats")

        atac_old_match = atac_prewie.find('span', class_="size10")
        lister.append(atac_old_match.text)

        atac_table_text = atac_table.find_all('td')[1:6]
        for j in (atac_table_text):
            lister.append(float(j.text))
            
        #Коэфиценты
        coeficent_all = soup.find_all("div",class_="block_body_nopadding")[3]
        coficient = coeficent_all.find_all('div',class_='odds_item odds_logo')

        def cof_func(coficient):
            list_coef = list()
            for i in coficient:
                odds_coeff = i.find_all('div',class_='odds_coeff')
                for j in odds_coeff:
                    if j == '\n':
                        continue
                    if j == '\xa0':
                        list_coef.clear()
                        break
                    list_coef.append(float(j.text))
                if len(list_coef) == 5:
                    return  list_coef  
                
        coficient_result = cof_func(coficient)
        lister.extend(coficient_result)
        df.loc[len(df)] = lister
    except Exception:  #заносим эту ошибку в переменную e
        continue
df

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,xG1,xG2,Удары1,...,Пголы2,Пза матч2,Ппобед2,Пничьи2,Ппорож2,поб1,X,поб2,ТМ 2.5,ТБ 2.5
0,1736825,"Бундеслига, 1-й тур, регулярный сезон, 05.08.2...",05.08.2022 21:35,Айнтрахт Франкфурт,1,Бавария,6,1.74,4.31,8.0,...,15.0,3.0,2.0,2.0,1.0,6.50,5.40,1.51,2.92,1.36
1,1736826,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 19:30,Боруссия Дортмунд,1,Байер,0,1.91,1.45,13.0,...,11.0,2.2,2.0,1.0,2.0,2.13,4.10,3.45,2.73,1.41
2,1736827,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Унион Берлин,3,Герта,1,1.45,0.56,18.0,...,7.0,1.4,1.0,0.0,4.0,1.87,3.72,4.30,1.92,1.94
3,1736828,"Бундеслига, 1-й тур, регулярный сезон, 07.08.2...",07.08.2022 18:30,Кёльн,3,Шальке,1,4.60,0.38,30.0,...,12.0,2.4,2.0,2.0,1.0,2.08,3.64,3.58,2.05,1.70
4,1736829,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Боруссия М,3,Хоффенхайм,1,2.32,0.17,18.0,...,11.0,2.2,4.0,1.0,0.0,2.07,3.78,3.48,2.73,1.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,1737118,"Бундеслига, 33-й тур, регулярный сезон, 21.05....",21.05.2023 18:30,Аугсбург,0,Боруссия Дортмунд,3,0.52,4.55,4.0,...,19.0,3.8,3.0,2.0,0.0,6.20,5.60,1.39,3.55,1.28
274,1737119,"Бундеслига, 33-й тур, регулярный сезон, 20.05....",20.05.2023 16:30,Герта,1,Бохум,1,1.47,1.42,14.0,...,6.0,1.2,1.0,2.0,2.0,2.36,3.70,2.70,2.42,1.55
275,1737120,"Бундеслига, 33-й тур, регулярный сезон, 20.05....",20.05.2023 16:30,Шальке,2,Айнтрахт Франкфурт,2,1.00,1.81,11.0,...,8.0,1.6,2.0,1.0,2.0,2.75,3.70,2.40,2.33,1.58
276,1737121,"Бундеслига, 33-й тур, регулярный сезон, 20.05....",20.05.2023 16:30,Вердер,1,Кёльн,1,1.64,1.17,13.0,...,11.0,2.2,3.0,1.0,1.0,2.46,3.65,2.65,2.22,1.64


1737122

In [68]:
df['счет1'].value_counts()
df['счет2'].value_counts()

1    94
0    73
2    65
3    28
4    10
6     3
7     2
5     2
6     1
Name: счет2, dtype: int64

In [69]:
df['Признак погоды'].value_counts()

солнечно                 244
переменная облачность     34
Name: Признак погоды, dtype: int64

In [72]:
#df.to_csv('data/table_matches_22.csv')